<a href="https://colab.research.google.com/github/Imranzr/EEG_Based_Auditory_Brain_Computer_Interface_Using_Deep_Learning_and_Machine_Learning_Model/blob/main/Auditory_BCI_Sliding_and_Overlapping_Window_epoching_based_Machine_Learning(ML)_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [1]:
pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 24.5 MB/s eta 0:00:00


In [2]:
pip install pyriemann

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 1.2 MB/s eta 0:00:00


In [5]:
import os
import argparse
import sys
import mne
import math
import time
import json
import numpy as np
from scipy.signal import butter, filtfilt
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, matthews_corrcoef

start = time.time()


def is_notebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True
        elif shell == 'TerminalInteractiveShell':
            return False
        else:
            return False
    except NameError:
        return False

if is_notebook():
    args = argparse.Namespace(s=None, c=None)
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('-s', default=None)
    parser.add_argument('-c', default=None, type=int)

    args = parser.parse_args(args=[])

print(args.s)
print(args.c)
print(__doc__)

subject = 'sub-B'
if args.s is not None:
    subject = args.s
test_class =1
if args.c is not None:
    test_class = args.c

import numpy as np
fnum = np.array([[1,4],
                 [2,5],
                 [3,6]])

trig_id = [2,8,32]
tasks = ['low', 'low', 'mid', 'mid', 'high', 'high']
reject={'eeg':100e-6,'eog':500e-6}

import os
import json
repository_base = os.path.dirname(os.path.dirname(os.path.abspath('/content/drive/MyDrive/dataverse_files/results/sub-B_classification_scores.json')))

#repository_base = os.path.dirname(os.path.dirname(os.path.abspath('file_path')))
base = os.path.join(repository_base, "eeg")
save_base = os.path.join(repository_base, "results")
if not os.path.exists(save_base):
    os.makedirs(save_base)

Fs = 1000
fc = [1, 40]
resample = None

from scipy.signal import butter, filtfilt
def apply_filter(data, b, a):
    r = filtfilt(b=b, a=a, x=data)
    return r
b,a = butter(N = 2, Wn = np.array(fc)/(Fs/2), btype = 'bandpass', output = 'ba')

t_file = []
nt_file = []
target_file = []
non_target_file = []

for i in range(len(fnum.ravel())):
    fname = os.path.join(base, subject, "eeg", "%s_task-%s_run-%d_eeg.vhdr" % (subject, tasks[i], fnum.ravel()[i]))
    print(fname)
    if np.any(fnum[test_class-1] == fnum.ravel()[i]):
        if isinstance(target_file, list):
            target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            target_file = target_file.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            t_file.append(fnum.ravel()[i])
        else:
            tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            tmp = tmp.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            target_file = mne.concatenate_raws([target_file, tmp])
            t_file.append(fnum.ravel()[i])
    else:
        if isinstance(non_target_file, list):
            non_target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            non_target_file = non_target_file.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            nt_file.append(fnum.ravel()[i])
        else:
            tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            tmp = tmp.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            non_target_file = mne.concatenate_raws([non_target_file, tmp])
            nt_file.append(fnum.ravel()[i])
if resample is not None:
    target_file.resample(resample)
    non_target_file.resample(resample)

if resample != None:
    target_file.resample(resample)
    non_target_file.resample(resample)

event_id={'target_stimulus_id':-100,'non_target_stimulus_id':-500}
target_eve = mne.events_from_annotations(target_file)
target_eve = mne.merge_events(target_eve[0],[trig_id[test_class-1]],event_id['target_stimulus_id'],replace_events=True)

non_target_eve = mne.events_from_annotations(non_target_file)
non_target_eve = mne.merge_events(non_target_eve[0],[trig_id[test_class-1]],event_id['non_target_stimulus_id'],replace_events=True)

tmin,tmax= -0.02, 1.4
baseline=(0.0,0.01)
target_epochs = mne.Epochs(target_file,events=target_eve, event_id=event_id['target_stimulus_id'], tmin=tmin,tmax=tmax, baseline=baseline, reject=reject,preload = True)

None
None
Automatically created module for IPython interactive environment
/content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-low_run-1_eeg.vhdr
Extracting parameters from /content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-low_run-1_eeg.vhdr...
Setting channel info structure...


<ipython-input-5-fdbca35346cd>:91: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
<ipython-input-5-fdbca35346cd>:91: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


Reading 0 ... 317919  =      0.000 ...   317.919 secs...
/content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-low_run-4_eeg.vhdr
Extracting parameters from /content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-low_run-4_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 312879  =      0.000 ...   312.879 secs...


<ipython-input-5-fdbca35346cd>:95: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
<ipython-input-5-fdbca35346cd>:95: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


/content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-mid_run-2_eeg.vhdr
Extracting parameters from /content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-mid_run-2_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 314879  =      0.000 ...   314.879 secs...


...

/content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-high_run-3_eeg.vhdr
Extracting parameters from /content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-high_run-3_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 319199  =      0.000 ...   319.199 secs...


<ipython-input-5-fdbca35346cd>:105: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
<ipython-input-5-fdbca35346cd>:105: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


/content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-high_run-6_eeg.vhdr
Extracting parameters from /content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-high_run-6_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 321519  =      0.000 ...   321.519 secs...


<ipython-input-5-fdbca35346cd>:105: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
<ipython-input-5-fdbca35346cd>:105: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  4', 'Stimulus/S  8', 'Stimulus/S 16', 'Stimulus/S 32', 'Stimulus/S129', 'Stimulus/S132', 'Stimulus/S144']
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  4', 'Stimulus/S  8', 'Stimulus/S 16', 'Stimulus/S 32', 'Stimulus/S129', 'Stimulus/S132', 'Stimulus/S144']
Not setting metadata
119 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 119 events and 1421 original time points ...
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epo

# Sliding and Overlapping Window Epoching

In [6]:

import numpy as np
def find_max_peak_window(data, Fs, window_size_ms=500, overlap_size_ms=100, epoch_index=0):
    """
    Function to find the window with the maximum peak value within a specific epoch and return its start and end indices.
    """
    window_size_samples = int(window_size_ms * Fs / 1000)
    overlap_size_samples = int(overlap_size_ms * Fs / 1000)
    epoch_data = data[epoch_index]
    n_channels, n_times = epoch_data.shape
    max_peak_value = -np.inf
    max_peak_start_index = None
    max_peak_end_index = None
    start = 0
    while start + window_size_samples <= n_times:
        window_data = epoch_data[:, start:start + window_size_samples]
        current_peak_value = np.max(window_data)
        if current_peak_value > max_peak_value:
            max_peak_value = current_peak_value
            max_peak_start_index = start
            max_peak_end_index = start + window_size_samples - 1
        start += (window_size_samples - overlap_size_samples)
    print(f"Epoch {epoch_index + 1} - Max Peak Value: {max_peak_value}")
    print(f"Epoch {epoch_index + 1} - Window Start Index: {max_peak_start_index}, End Index: {max_peak_end_index}")
    return max_peak_start_index, max_peak_end_index

def analyze_all_epochs_max_peaks(data, Fs, window_size_ms=500, overlap_size_ms=100):
    """
    Function to find and print the maximum peak values and corresponding window indices across all epochs.
    """
    n_epochs = data.shape[0]
    peak_window_indices = []
    for epoch_index in range(n_epochs):
        print(f"\nAnalyzing Epoch {epoch_index + 1}")
        start_index, end_index = find_max_peak_window(data, Fs, window_size_ms, overlap_size_ms, epoch_index)
        peak_window_indices.append((start_index, end_index))
    return peak_window_indices

def re_epoch_data(data, peak_window_indices):
    """
    Function to re-epoch the data based on the peak window indices.
    """
    re_epoched_data = []
    for epoch_index, (start, end) in enumerate(peak_window_indices):
        re_epoched_epoch_data = data[epoch_index][:, start:end + 1]
        re_epoched_data.append(re_epoched_epoch_data)
    re_epoched_data = np.array(re_epoched_data)
    print("\nRe-epoching complete.")
    print("Re-epoched data shape:", re_epoched_data.shape)
    return re_epoched_data

Fs = 1000
data = target_epochs.get_data()
peak_window_indices = analyze_all_epochs_max_peaks(data, Fs, window_size_ms=500, overlap_size_ms=100)
re_epoched_data = re_epoch_data(data, peak_window_indices)

from mne import EpochsArray
from mne import create_info
n_channels = re_epoched_data.shape[1]
info = create_info(ch_names=[f'chan{i}' for i in range(n_channels)], sfreq=Fs, ch_types='eeg')
re_epoched_epochs = EpochsArray(re_epoched_data, info)
print("Re-epoched data converted to MNE Epochs:", re_epoched_epochs)



from mne import EpochsArray, create_info
n_channels = re_epoched_data.shape[1]
info = create_info(ch_names=[f'chan{i}' for i in range(n_channels)], sfreq=Fs, ch_types='eeg')
re_epoched_epochs = EpochsArray(re_epoched_data, info)
baseline=(0.0,0.01)
re_epoched_epochs.apply_baseline(baseline)
re_epoched_epochs.filter(l_freq=0.50, h_freq=100.00)
print("Re-epoched data converted to MNE Epochs and processed with baseline correction and filtering.")

tmin,tmax= -0.0, 0.499
baseline=(0.0,0.01)
non_target_epochs = mne.Epochs(non_target_file, events=non_target_eve, event_id=event_id['non_target_stimulus_id'], tmin=tmin,tmax=tmax, baseline=baseline, reject=reject,preload = True)


import mne
tmin, tmax = -0.0, 0.499
baseline=(0.0,0.01)
non_target_epochs = mne.Epochs(
    non_target_file,
    events=non_target_eve,
    event_id=event_id['non_target_stimulus_id'],
    tmin=tmin,
    tmax=tmax,
    baseline=baseline,
    reject=reject,
    preload=True
)

new_channel_names = [f'chan{i}' for i in range(len(non_target_epochs.ch_names))]
rename_dict = dict(zip(non_target_epochs.ch_names, new_channel_names))
non_target_epochs.rename_channels(rename_dict)
print("Updated channel names:", non_target_epochs.ch_names)




Analyzing Epoch 1
Epoch 1 - Max Peak Value: 5.3609090909090905e-05
Epoch 1 - Window Start Index: 800, End Index: 1299

Analyzing Epoch 2
Epoch 2 - Max Peak Value: 2.9702295397418008e-05
Epoch 2 - Window Start Index: 0, End Index: 499

Analyzing Epoch 3
Epoch 3 - Max Peak Value: 1.9700354081974367e-05
Epoch 3 - Window Start Index: 800, End Index: 1299

Analyzing Epoch 4
Epoch 4 - Max Peak Value: 1.8520261664344037e-05
Epoch 4 - Window Start Index: 0, End Index: 499

Analyzing Epoch 5
Epoch 5 - Max Peak Value: 2.7344734807680046e-05
Epoch 5 - Window Start Index: 0, End Index: 499

Analyzing Epoch 6
Epoch 6 - Max Peak Value: 1.838181818181818e-05
Epoch 6 - Window Start Index: 400, End Index: 899

Analyzing Epoch 7
Epoch 7 - Max Peak Value: 1.841774308675113e-05
Epoch 7 - Window Start Index: 400, End Index: 899

Analyzing Epoch 8
Epoch 8 - Max Peak Value: 1.8918203042186065e-05
Epoch 8 - Window Start Index: 400, End Index: 899

Analyzing Epoch 9
Epoch 9 - Max Peak Value: 1.552001359962160

<ipython-input-6-2346cbd8a145>:71: RuntimeWarning: filter_length (6601) is longer than the signal (500), distortion is likely. Reduce filter length or filter a longer signal.
  re_epoched_epochs.filter(l_freq=0.50, h_freq=100.00)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:    0.8s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:    1.2s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:    1.4s
[Parallel(n_jobs=1)]: Done 3041 t

Re-epoched data converted to MNE Epochs and processed with baseline correction and filtering.
Not setting metadata
237 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 237 events and 500 original time points ...


[Parallel(n_jobs=1)]: Done 3527 tasks      | elapsed:    2.0s


    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp2', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp2', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7

In [7]:
epochs = mne.concatenate_epochs([re_epoched_epochs, non_target_epochs])
epochs = epochs.copy().pick_types(eeg=True, eog=False)

Not setting metadata
246 matching events found
Applying baseline correction (mode: mean)
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


<ipython-input-7-e84d836bf4c9>:1: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([re_epoched_epochs, non_target_epochs])


# Logistic Regression

In [8]:

clf = make_pipeline(XdawnCovariances(8),
                    TangentSpace(metric='logeuclid'),
                    LogisticRegression( penalty='l1', solver='liblinear', multi_class='ovr'))

epochs_data = epochs.get_data()
labels = epochs.events[:, -1]
preds = np.zeros(len(labels))

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

preds = np.empty(len(labels))
for train_idx, test_idx in cv.split(epochs_data, labels):
    clf.fit(epochs_data[train_idx], labels[train_idx])
    preds[test_idx] = clf.predict(epochs_data[test_idx])

report = classification_report(labels, preds, target_names=['non-target', 'target'], output_dict=True)
print(report)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was depre

{'non-target': {'precision': 0.983957219251337, 'recall': 0.9735449735449735, 'f1-score': 0.9787234042553191, 'support': 189.0}, 'target': {'precision': 0.9152542372881356, 'recall': 0.9473684210526315, 'f1-score': 0.9310344827586207, 'support': 57.0}, 'accuracy': 0.967479674796748, 'macro avg': {'precision': 0.9496057282697363, 'recall': 0.9604566972988025, 'f1-score': 0.9548789435069699, 'support': 246.0}, 'weighted avg': {'precision': 0.9680382356257171, 'recall': 0.967479674796748, 'f1-score': 0.967673532201206, 'support': 246.0}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


#Support Vector Machine (SVM)

In [9]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

clf = make_pipeline(XdawnCovariances(8),
                    TangentSpace(metric='logeuclid'),
                    SVC(kernel='linear', C=1, decision_function_shape='ovr'))


epochs_data = epochs.get_data()
labels = epochs.events[:, -1]
preds = np.zeros(len(labels))

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

train_accuracies = []
test_accuracies = []
train_losses = []
test_losses = []
preds = np.empty(len(labels))
for train_idx, test_idx in cv.split(epochs_data, labels):
    clf.fit(epochs_data[train_idx], labels[train_idx])
    train_preds = clf.predict(epochs_data[train_idx])
    test_preds = clf.predict(epochs_data[test_idx])
    train_accuracy = accuracy_score(labels[train_idx], train_preds)
    test_accuracy = accuracy_score(labels[test_idx], test_preds)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    train_loss = np.mean(train_preds != labels[train_idx])
    test_loss = np.mean(test_preds != labels[test_idx])
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    preds[test_idx] = test_preds
report = classification_report(labels, preds, target_names=['non-target', 'target'], output_dict=True)
print(report)





{'non-target': {'precision': 0.958974358974359, 'recall': 0.9894179894179894, 'f1-score': 0.9739583333333334, 'support': 189.0}, 'target': {'precision': 0.9607843137254902, 'recall': 0.8596491228070176, 'f1-score': 0.9074074074074074, 'support': 57.0}, 'accuracy': 0.959349593495935, 'macro avg': {'precision': 0.9598793363499246, 'recall': 0.9245335561125034, 'f1-score': 0.9406828703703705, 'support': 246.0}, 'weighted avg': {'precision': 0.9593937387337673, 'recall': 0.959349593495935, 'f1-score': 0.9585379968383018, 'support': 246.0}}


# Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier

clf = make_pipeline(XdawnCovariances(8),
                    TangentSpace(metric='logeuclid'),
                    RandomForestClassifier(n_estimators=100, random_state=42))

epochs_data = epochs.get_data()
labels = epochs.events[:, -1]
preds = np.zeros(len(labels))

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

train_accuracies = []
test_accuracies = []
train_losses = []
test_losses = []

preds = np.empty(len(labels))
for train_idx, test_idx in cv.split(epochs_data, labels):

    clf.fit(epochs_data[train_idx], labels[train_idx])
    train_preds = clf.predict(epochs_data[train_idx])
    test_preds = clf.predict(epochs_data[test_idx])
    train_accuracy = accuracy_score(labels[train_idx], train_preds)
    test_accuracy = accuracy_score(labels[test_idx], test_preds)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    train_loss = np.mean(train_preds != labels[train_idx])
    test_loss = np.mean(test_preds != labels[test_idx])

    train_losses.append(train_loss)
    test_losses.append(test_loss)
    preds[test_idx] = test_preds

report = classification_report(labels, preds, target_names=['non-target', 'target'], output_dict=True)
print(report)




{'non-target': {'precision': 0.875, 'recall': 1.0, 'f1-score': 0.9333333333333333, 'support': 189.0}, 'target': {'precision': 1.0, 'recall': 0.5263157894736842, 'f1-score': 0.6896551724137931, 'support': 57.0}, 'accuracy': 0.8902439024390244, 'macro avg': {'precision': 0.9375, 'recall': 0.763157894736842, 'f1-score': 0.8114942528735632, 'support': 246.0}, 'weighted avg': {'precision': 0.9039634146341463, 'recall': 0.8902439024390244, 'f1-score': 0.8768713204373423, 'support': 246.0}}


# Polynomial Logistic Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression

clf = make_pipeline(XdawnCovariances(8),
                    TangentSpace(metric='logeuclid'),
                    PolynomialFeatures(degree=2, include_bias=False),
                    LogisticRegression(penalty='l2', solver='liblinear', multi_class='ovr'))

epochs_data = epochs.get_data()
labels = epochs.events[:, -1]
preds = np.zeros(len(labels))

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
train_accuracies = []
test_accuracies = []
train_losses = []
test_losses = []

preds = np.empty(len(labels))
for train_idx, test_idx in cv.split(epochs_data, labels):
    clf.fit(epochs_data[train_idx], labels[train_idx])
    train_preds = clf.predict(epochs_data[train_idx])
    test_preds = clf.predict(epochs_data[test_idx])
    train_accuracy = accuracy_score(labels[train_idx], train_preds)
    test_accuracy = accuracy_score(labels[test_idx], test_preds)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    train_loss = np.mean(train_preds != labels[train_idx])
    test_loss = np.mean(test_preds != labels[test_idx])
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    preds[test_idx] = test_preds


report = classification_report(labels, preds, target_names=['non-target', 'target'], output_dict=True)
print(report)




In [ ]:
# SVM_Plt

epochs_range = range(1, len(train_accuracies) + 1)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
ax1.plot(epochs_range, train_accuracies, label='Train Accuracy', marker='o')
ax1.plot(epochs_range, test_accuracies, label='Test Accuracy', marker='x')
ax1.set_title('Train and Test Accuracy')
ax1.set_xlabel('Fold')
ax1.set_ylabel('Accuracy')
ax1.legend()


for i in range(len(train_accuracies)):
    ax1.annotate(f'{train_accuracies[i]:.2f}', (epochs_range[i], train_accuracies[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
    ax1.annotate(f'{test_accuracies[i]:.2f}', (epochs_range[i], test_accuracies[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)

ax2.plot(epochs_range, train_losses, label='Train Loss', marker='o')
ax2.plot(epochs_range, test_losses, label='Test Loss', marker='x')
ax2.set_title('Train and Test Loss')
ax2.set_xlabel('Fold')
ax2.set_ylabel('Loss')
ax2.legend()

for i in range(len(train_losses)):
    ax2.annotate(f'{train_losses[i]:.2f}', (epochs_range[i], train_losses[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
    ax2.annotate(f'{test_losses[i]:.2f}', (epochs_range[i], test_losses[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)

plt.tight_layout()
plt.show()
cm = confusion_matrix(labels, preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['non-target', 'target'], yticklabels=['non-target', 'target'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()




#Random_forest_plt

epochs_range = range(1, len(train_accuracies) + 1)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

ax1.plot(epochs_range, train_accuracies, label='Train Accuracy', marker='o')
ax1.plot(epochs_range, test_accuracies, label='Test Accuracy', marker='x')
ax1.set_title('Train and Test Accuracy')
ax1.set_xlabel('Fold')
ax1.set_ylabel('Accuracy')
ax1.legend()


for i in range(len(train_accuracies)):
    ax1.annotate(f'{train_accuracies[i]:.2f}', (epochs_range[i], train_accuracies[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
    ax1.annotate(f'{test_accuracies[i]:.2f}', (epochs_range[i], test_accuracies[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)

ax2.plot(epochs_range, train_losses, label='Train Loss', marker='o')
ax2.plot(epochs_range, test_losses, label='Test Loss', marker='x')
ax2.set_title('Train and Test Loss')
ax2.set_xlabel('Fold')
ax2.set_ylabel('Loss')
ax2.legend()

for i in range(len(train_losses)):
    ax2.annotate(f'{train_losses[i]:.2f}', (epochs_range[i], train_losses[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
    ax2.annotate(f'{test_losses[i]:.2f}', (epochs_range[i], test_losses[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)

plt.tight_layout()
plt.show()

cm = confusion_matrix(labels, preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['non-target', 'target'], yticklabels=['non-target', 'target'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# plt_poly_l_Reg
epochs_range = range(1, len(train_accuracies) + 1)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

ax1.plot(epochs_range, train_accuracies, label='Train Accuracy', marker='o')
ax1.plot(epochs_range, test_accuracies, label='Test Accuracy', marker='x')
ax1.set_title('Train and Test Accuracy')
ax1.set_xlabel('Fold')
ax1.set_ylabel('Accuracy')
ax1.legend()

for i in range(len(train_accuracies)):
    ax1.annotate(f'{train_accuracies[i]:.2f}', (epochs_range[i], train_accuracies[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
    ax1.annotate(f'{test_accuracies[i]:.2f}', (epochs_range[i], test_accuracies[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
ax2.plot(epochs_range, train_losses, label='Train Loss', marker='o')
ax2.plot(epochs_range, test_losses, label='Test Loss', marker='x')
ax2.set_title('Train and Test Loss')
ax2.set_xlabel('Fold')
ax2.set_ylabel('Loss')
ax2.legend()


for i in range(len(train_losses)):
    ax2.annotate(f'{train_losses[i]:.2f}', (epochs_range[i], train_losses[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
    ax2.annotate(f'{test_losses[i]:.2f}', (epochs_range[i], test_losses[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
plt.tight_layout()
plt.show()


cm = confusion_matrix(labels, preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['non-target', 'target'], yticklabels=['non-target', 'target'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()